# Google Analytics Customer Revenue Prediction


La règle des 80/20 s'est avérée vraie pour de nombreuses entreprises - seul un faible pourcentage de clients produit la majeure partie des revenus. Ainsi, les équipes de marketing sont mises au défi de faire les investissements appropriés dans les stratégies promotionnelles.
Dans ce problème, nous sommes mis au défi d'analyser un ensemble de données client Google Merchandise Store (également connu sous le nom de GStore, où Google swag est vendu) pour prédire les revenus par client.

L’objectif de ce projet était de prédire la somme des revenues de google par utilisateur.


### Data Fields
- fullVisitorId- Identifiant unique pour chaque utilisateur du Google Merchandise Store.
- channelGrouping - Chaîne par laquelle les utilisateurs passent pour accéder au Store.
- date - Date à laquelle l’utilisateur a visité le store.
- device - Spécificités de l’appareil que l’utilisateur a utilisé pour accéder au store.
- geoNetwork - Cette section contient les informations géographiques de l’utilisateur.
- sessionId - Un identifiant unique pour cette visite.
- socialEngagementType - Type d’engagement, que ce soit un “engagé socialement” ou “Pas engagé socialement”.
- totals - Cette section contient les valeurs agrégées de la session.
- trafficSource - Cette section contient des informations sur la source de trafic à l'origine de la session.
- visitId -Un identifiant pour cette session. Cela fait partie de la valeur généralement stockée en tant que cookie.
- visitNumber -  Numéro de session de cet utilisateur. S'il s'agit de la première session, celle-ci est définie sur 1.
- visitStartTime -  timestamp

### Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import datetime as datetime
from datetime import timedelta, date
import seaborn as sns
import matplotlib.cm as CM
from pandas.io.json import json_normalize
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split
%matplotlib inline
import math
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import pyplot as plt

/Users/nathanamar/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning:

Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.



In [2]:
def load_df(path):
    df = pd.read_csv(path, dtype={'fullVisitorId': 'str'})
    json_columns = ['totals','trafficSource','device','geoNetwork']
    for json_col in json_columns: 
        in_df = pd.DataFrame(df.pop(json_col).apply(pd.io.json.loads).values.tolist(), index=df.index)
        df = df.join(in_df) 
     
    #json columns of columns
    in_df = pd.DataFrame(df.pop('adwordsClickInfo').values.tolist(), index=df.index)
    df = df.join(in_df)     
    return df

## Importation et analyse du train

In [3]:
train = load_df("train.csv")

FileNotFoundError: [Errno 2] File b'train.csv' does not exist: b'train.csv'

In [ ]:
train.shape

In [ ]:
train.columns

In [ ]:
train.head()

In [ ]:
print(train.info())

## Analyse et importation du test

In [ ]:
test = load_df("test.csv")

In [ ]:
test.shape

In [ ]:
test.columns

In [ ]:
print(test.info())

In [ ]:
test.head()

Comme on l'a vu avec le head il semble il y avoir bcp de NA, il serait interessant de savoir ou et combien

In [ ]:
train.isna().sum()

**Faisons un plot en pourcentage répresentant combien et ou il y a des NA**


In [ ]:
miss_per = {}
for k, v in dict(train.isna().sum(axis=0)).items():
    if v == 0:
        continue
    miss_per[k] = 100 * float(v) / len(train)
    
import operator 
sorted_x = sorted(miss_per.items(), key=operator.itemgetter(1), reverse=True)
print ("Il y a " + str(len(miss_per)) + " colonnes avec des valeurs manquantes")

kys = [_[0] for _ in sorted_x][::-1]
vls = [_[1] for _ in sorted_x][::-1]
trace1 = go.Bar(y = kys, orientation="h" , x = vls, marker=dict(color="#d6a5ff"))
layout = go.Layout(title="Pourcentage de valeur manquante", 
                   xaxis=dict(title="Pourcentage"), 
                   height=400, margin=dict(l=300, r=300))
figure = go.Figure(data = [trace1], layout = layout)
iplot(figure)

Remplacons tout les champs vide avec des NaN

In [ ]:

train.replace('(not set)', np.nan)
train.replace("not available in demo dataset", np.nan)
train.replace('(not provided)', np.nan)
train.replace('unknown.unknown', np.nan)
train.replace('(none)', np.nan)
train.replace('/', np.nan)
train.replace('Not Socially Engaged', np.nan)
test.replace('(not set)', np.nan)
test.replace("not available in demo dataset", np.nan)
test.replace('(not provided)', np.nan)
test.replace('unknown.unknown', np.nan)
test.replace('(none)', np.nan)
test.replace('/', np.nan)
test.replace('Not Socially Engaged', np.nan)

Jetons un oeil au variable numeric du train et test set

- train

In [ ]:
numeric_features_train = train.select_dtypes(include=[np.number])
numeric_features_train.columns

- test

In [ ]:
numeric_features_test = test.select_dtypes(include=[np.number])
numeric_features_test.columns

Jetons un oeil au variable catégorielle du train et test set

- train

In [ ]:
categorical_features_train = train.select_dtypes(include=[np.object])
categorical_features_train.columns

- test

In [ ]:

categorical_features_test = test.select_dtypes(include=[np.object])
categorical_features_test.columns

## Exploration des données et Analyses (EDA)

In [ ]:
train.columns

In [ ]:
device_cols = ["browser", "deviceCategory", "operatingSystem"]

colors = ["#d6a5ff", "#fca6da", "#f4d39c", "#a9fcca"]
traces = []
for i, col in enumerate(device_cols):
    t = train[col].value_counts()
    traces.append(go.Bar(marker=dict(color=colors[i]),orientation="h", y = t.index[:15][::-1], x = t.values[:15][::-1]))

fig = tools.make_subplots(rows=1, cols=3, subplot_titles=["Visites: Categorie", "Visites: Navigateur","Visites: OS"], print_grid=False)
fig.append_trace(traces[1], 1, 1)
fig.append_trace(traces[0], 1, 2)
fig.append_trace(traces[2], 1, 3)

fig['layout'].update(height=400, showlegend=False, title="Visites par attributs de périphérique")
iplot(fig)

## convert transaction revenue to float
train["transactionRevenue"] = train["transactionRevenue"].astype('float')

device_cols = ["browser", "deviceCategory", "operatingSystem"]

fig = tools.make_subplots(rows=1, cols=3, subplot_titles=["Revenu moyen: Categorie", "Revenu moyen: Navigateur","Revenu moyen: OS"], print_grid=False)

colors = ["red", "green", "purple"]
trs = []
for i, col in enumerate(device_cols):
    tmp = train.groupby(col).agg({"transactionRevenue": "mean"}).reset_index().rename(columns={"transactionRevenue" : "Mean Revenue"})
    tmp = tmp.dropna().sort_values("Mean Revenue", ascending = False)
    tr = go.Bar(x = tmp["Mean Revenue"][::-1], orientation="h", marker=dict(opacity=0.5, color=colors[i]), y = tmp[col][::-1])
    trs.append(tr)

fig.append_trace(trs[1], 1, 1)
fig.append_trace(trs[0], 1, 2)
fig.append_trace(trs[2], 1, 3)
fig['layout'].update(height=400, showlegend=False, title="Revenu moyen par attribut d'appareil")
iplot(fig)


Il y a une différence significative dans les visites depuis les mobiles et les tablettes, mais les revenus moyens pour les deux sont très proches. Il est intéressant de noter que le nombre maximal de visites provient du navigateur Chrome, mais que les revenus maximaux sont collectés grâce aux visites via Firefox. Les utilisateurs de Chrome OS ont généré des revenus maximaux, bien que le nombre maximal de visites proviennent d'utilisateurs Windows et Macintosh.

### Geo Network Attributes:

Voyons quels sont tous les attributs de geoNetwork.

Analysons la répartition des utilisateurs sur 5 continents.



In [ ]:
train.columns

In [ ]:
geo_cols = ['city', 'continent','country',
            'metro', 'networkDomain', 'region','subContinent']
geo_cols = ['continent','subContinent']

colors = ["#d6a5ff", "#fca6da"]
fig = tools.make_subplots(rows=1, cols=2, subplot_titles=["Visites : GeoNetwork Continent", "Visites : GeoNetwork subContinent"], print_grid=False)
trs = []
for i,col in enumerate(geo_cols):
    t = train[col].value_counts()
    tr = go.Bar(x = t.index[:20], marker=dict(color=colors[i]), y = t.values[:20])
    trs.append(tr)

fig.append_trace(trs[0], 1, 1)
fig.append_trace(trs[1], 1, 2)
fig['layout'].update(height=400, margin=dict(b=150), showlegend=False)
iplot(fig)




geo_cols = ['continent','subContinent']
fig = tools.make_subplots(rows=1, cols=2, subplot_titles=["Revenu moyen: Continent", "Revenu moyen: SubContinent"], print_grid=False)

colors = ["blue", "orange"]
trs = []
for i, col in enumerate(geo_cols):
    tmp = train.groupby(col).agg({"transactionRevenue": "mean"}).reset_index().rename(columns={"transactionRevenue" : "Mean Revenue"})
    tmp = tmp.dropna().sort_values("Mean Revenue", ascending = False)
    tr = go.Bar(y = tmp["Mean Revenue"], orientation="v", marker=dict(opacity=0.5, color=colors[i]), x= tmp[col])
    trs.append(tr)

fig.append_trace(trs[0], 1, 1)
fig.append_trace(trs[1], 1, 2)
fig['layout'].update(height=450, margin=dict(b=200), showlegend=False)
iplot(fig)

On peut voir avec une meilleur visualisation

In [ ]:
colorscale = [[0, 'rgb(102,194,165)'], [0.0005, 'rgb(102,194,165)'], 
              [0.01, 'rgb(171,221,164)'], [0.02, 'rgb(230,245,152)'], 
              [0.04, 'rgb(255,255,191)'], [0.05, 'rgb(254,224,139)'], 
              [0.10, 'rgb(253,174,97)'], [0.25, 'rgb(213,62,79)'], [1.0, 'rgb(158,1,66)']]

data = [ dict(
        type = 'choropleth',
        autocolorscale = False,
        colorscale = colorscale,
        showscale = True,
        locations = train["country"].value_counts().index,
        locationmode = 'country names',
        z = train["country"].value_counts().values,
        marker = dict(
            line = dict(color = 'rgb(250,250,225)', width = 1)),
            colorbar = dict( title = 'Customer Visits ')
            ) 
       ]

layout = dict(
    height=600,
    title = 'Distribution mondiale des visites clients',
    geo = dict(
        showframe = True,
        showocean = True,
        oceancolor = 'rgb(28,107,160)',
        projection = dict(
        type = 'orthographic',
            rotation = dict(
                    lon = 50,
                    lat = 10),
        ),
        lonaxis =  dict(
                showgrid = True,
                gridcolor = 'rgb(12, 102, 102)'
            ),
        lataxis = dict(
                showgrid = True,
                gridcolor = 'rgb(12, 102, 102)'
                )
            ),
        )
fig = dict(data=data, layout=layout)
iplot(fig)

On peut voir sur le graph que les plus grandes visites se font aux USA 

### Channel grouping

#### Channel: The channel via which the user came to the Store.

In [ ]:
train.channelGrouping.value_counts().plot(kind="bar",title="channelGrouping distribution",figsize=(8,8),rot=25,colormap='Paired')

In [ ]:
tmp = train["channelGrouping"].value_counts()
colors = ["#8d44fc", "#ed95d5", "#caadf7", "#6161b7", "#7e7eba", "#babad1"]
trace = go.Pie(labels=tmp.index, values=tmp.values, marker=dict(colors=colors))
layout = go.Layout(title="Channel Grouping", height=400)
fig = go.Figure(data = [trace], layout = layout)
iplot(fig, filename='basic_pie_chart')

#### Convert date to year, month and day

In [ ]:

train['date'] = pd.to_datetime((train['date']).astype(str), format = '%Y-%m-%d')
train[['year', 'month', 'day']] = train['date'].astype(str).str.split('-', expand = True)    
train.drop('date', axis =1,inplace = True)

In [ ]:

test['date'] = pd.to_datetime((test['date']).astype(str), format = '%Y-%m-%d')
test[['year', 'month', 'day']] = test['date'].astype(str).str.split('-', expand = True)    
test.drop('date', axis =1,inplace = True)


In [ ]:
fig = tools.make_subplots(rows=1, cols=3, subplot_titles=["Visites par mois", "Visites par jour du mois"], print_grid=False)
trs = []
for i,col in enumerate(["month", "day"]):
    t = train[col].value_counts()
    tr = go.Bar(x = t.index, marker=dict(color=colors[i]), y = t.values)
    trs.append(tr)

fig.append_trace(trs[0], 1, 1)
fig.append_trace(trs[1], 1, 2)
fig['layout'].update(height=400, showlegend=False)
iplot(fig)



tmp1 = train.groupby('month').agg({"transactionRevenue" : "mean"}).reset_index()
tmp2 = train.groupby('day').agg({"transactionRevenue" : "mean"}).reset_index()

fig = tools.make_subplots(rows=1, cols=3, subplot_titles=["Revenu moyen par mois", "Revenu moyen par jour du mois"], print_grid=False)
tr1 = go.Bar(x = tmp1.month, marker=dict(color="red", opacity=0.5), y = tmp1.transactionRevenue)
tr2 = go.Bar(x = tmp2.day, marker=dict(color="orange", opacity=0.5), y = tmp2.transactionRevenue)
fig.append_trace(tr1, 1, 1)
fig.append_trace(tr2, 1, 2)
fig['layout'].update(height=400, showlegend=False)
iplot(fig)

#### Fréquence des visites

In [ ]:
vn = train["visitNumber"].value_counts()
def vn_bins(x):
    if x == 1:
        return "1" 
    elif x < 5:
        return "2-5"
    elif x < 10:
        return "5-10"
    elif x < 50:
        return "10-50"
    elif x < 100:
        return "50-100"
    else:
        return "100+"
    
vn = train["visitNumber"].apply(vn_bins).value_counts()

trace1 = go.Bar(y = vn.index[::-1], orientation="h" , x = vn.values[::-1], marker=dict(color="#7af9ad"))
layout = go.Layout(title="Distribution du nombre de visites", 
                   xaxis=dict(title="Frequence"),yaxis=dict(title="Nombre de visites") ,
                   height=400, margin=dict(l=300, r=300))
figure = go.Figure(data = [trace1], layout = layout)
iplot(figure)

In [ ]:
non_zero = train[train["transactionRevenue"] > 0]["transactionRevenue"]
print ("Il y a  " + str(len(non_zero)) + " visiteurs du train dataset ayant des revenus de transaction totaux non nuls")

plt.figure(figsize=(12,6))
sns.distplot(non_zero)
plt.title("Répartition des transactions totales non nulles");
plt.xlabel("Total Transactions");

There are 11515 visitors in the train dataset having non zero total transaction revenue


In [ ]:
plt.figure(figsize=(12,6))
sns.distplot(np.log1p(non_zero))
plt.title("Log - Répartition des transactions totales non nulle");
plt.xlabel("Log - Total Transactions");

# Attributs du profil d'un visiteur


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

agg_dict = {}
for col in ["bounces", "hits", "newVisits", "pageviews", "transactionRevenue"]:
    train[col] = train[col].astype('float')
    agg_dict[col] = "sum"
tmp = train.groupby("fullVisitorId").agg(agg_dict).reset_index()
tmp.head()

In [ ]:
train.columns

In [ ]:
def getbin_hits(x):
    if x < 5:
        return "1-5"
    elif x < 10:
        return "5-10"
    elif x < 30:
        return "10-30"
    elif x < 50:
        return "30-50"
    elif x < 100:
        return "50-100"
    else:
        return "100+"

    
#["bounces", "hits", "newVisits", "pageviews", "transactionRevenue"]:
    
    
tmp["total_hits_bin"] = tmp["hits"].apply(getbin_hits)
tmp["totals_bounces_bin"] = tmp["bounces"].apply(lambda x : str(x) if x <= 5 else "5+")
tmp["totals_pageviews_bin"] = tmp["pageviews"].apply(lambda x : str(x) if x <= 50 else "50+")

t1 = tmp["total_hits_bin"].value_counts()
t2 = tmp["totals_bounces_bin"].value_counts()
t3 = tmp["newVisits"].value_counts()
t4 = tmp["totals_pageviews_bin"].value_counts()

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=["Nombre total de visites par utilisateur", "Total de rebonds par utilisateur", 
                                                         "Nombre total de NewVistits par utilisateur", "Nombre total de pages vues par utilisateur"], print_grid=False)

tr1 = go.Bar(x = t1.index[:20], y = t1.values[:20])
tr2 = go.Bar(x = t2.index[:20], y = t2.values[:20])
tr3 = go.Bar(x = t3.index[:20], y = t3.values[:20])
tr4 = go.Bar(x = t4.index, y = t4.values)

fig.append_trace(tr1, 1, 1)
fig.append_trace(tr2, 1, 2)
fig.append_trace(tr3, 2, 1)
fig.append_trace(tr4, 2, 2)

fig['layout'].update(height=700, showlegend=False)
iplot(fig)

## Model
Identifions quelles colonnes peuvent être supprimées.
- Supression des colonnes avec des valeurs nulles
- Suppression des identifiants et autres colonnes non pertinentes

#### Supression de toutes les colonnes qui ont toutes les valeurs nulles


In [ ]:
train.dropna(axis=1, how='all')

In [ ]:
test.dropna(axis=1, how='all')

In [ ]:
#les hits et les pages vues sont les seules valeurs numériques, convertissons-les en objet numpy
train["hits"].fillna(0, inplace=True)
train["hits"] = train["hits"].astype('float')
train["pageviews"].fillna(0, inplace=True)
train["pageviews"] = train["pageviews"].astype('float')

In [ ]:
test["hits"].fillna(0, inplace=True)
test["hits"] = train["hits"].astype('float')
test["pageviews"].fillna(0, inplace=True)
test["pageviews"] = train["pageviews"].astype('float')

In [ ]:
train.shape,test.shape


Application du natural log sur transactionRevenue ce qui donne transactionRevenue_log


In [ ]:
train["transactionRevenue"].fillna(0, inplace=True)
train["transactionRevenue"] = train["transactionRevenue"].astype('float')
train['transactionRevenue_log'] = np.log(train[train['transactionRevenue'] > 0]["transactionRevenue"] + 0.01)


##### Copions le  dataframe dans des dataframes temporaires pour construire notre modele

In [ ]:
train_dataset = train.copy()
test_dataset = test.copy()

### Variable à predire

In [ ]:
y = train_dataset['transactionRevenue_log'].fillna(0)
y.head(5)

In [ ]:
y.sample(10)

Suppression des colonnes qui n'aident pas au training


In [ ]:
train_dataset.drop('fullVisitorId', axis=1, inplace=True)
train_dataset.drop('sessionId', axis=1, inplace=True)
train_dataset.drop('transactionRevenue', axis=1, inplace=True)
train_dataset.drop('visitId', axis=1, inplace=True)
train_dataset.drop('visitStartTime', axis=1, inplace=True)
train_dataset.drop('transactionRevenue_log', axis=1, inplace=True)
train_dataset.drop('visits', axis=1, inplace=True)
train_dataset.drop('visitNumber', axis=1, inplace=True)
train_dataset.drop('isTrueDirect', axis=1, inplace=True)
train_dataset.drop('adContent', axis=1, inplace=True)
train_dataset.drop('networkDomain', axis=1, inplace=True)
train_dataset.drop('isVideoAd', axis=1, inplace=True)
train_dataset.drop('gclId', axis=1, inplace=True)
train_dataset.drop('slot', axis=1, inplace=True)
train_dataset.drop('adNetworkType', axis=1, inplace=True)
train_dataset.drop('keyword', axis=1, inplace=True)
train_dataset.drop('referralPath', axis=1, inplace=True)
train_dataset.drop('page', axis=1, inplace=True)
train_dataset.drop('campaignCode', axis=1, inplace=True)
train_dataset.drop('newVisits', axis=1, inplace=True)
train_dataset.drop('bounces', axis=1, inplace=True)
train_dataset.drop('targetingCriteria', axis=1, inplace=True)
train_dataset.drop('socialEngagementType', axis=1, inplace=True)
train_dataset.drop('browserSize', axis=1, inplace=True)
train_dataset.drop('browserVersion', axis=1, inplace=True)
train_dataset.drop('flashVersion', axis=1, inplace=True)
train_dataset.drop('language', axis=1, inplace=True)
train_dataset.drop('mobileDeviceBranding', axis=1, inplace=True)
train_dataset.drop('mobileDeviceInfo', axis=1, inplace=True)
train_dataset.drop('mobileDeviceMarketingName', axis=1, inplace=True)
train_dataset.drop('mobileDeviceModel', axis=1, inplace=True)
train_dataset.drop('mobileInputSelector', axis=1, inplace=True)
train_dataset.drop('operatingSystemVersion', axis=1, inplace=True)
train_dataset.drop('screenColors', axis=1, inplace=True)
train_dataset.drop('screenResolution', axis=1, inplace=True)
train_dataset.drop('cityId', axis=1, inplace=True)
train_dataset.drop('longitude', axis=1, inplace=True)
train_dataset.drop('latitude', axis=1, inplace=True)
train_dataset.drop('networkLocation', axis=1, inplace=True)
train_dataset.drop('criteriaParameters', axis=1, inplace=True)



In [ ]:
train_dataset.head(5)


In [ ]:
train_dataset.shape

Suppression des colonnes qui n'aident pas au test


In [ ]:
test_dataset.drop('fullVisitorId', axis=1, inplace=True)
test_dataset.drop('sessionId', axis=1, inplace=True)
test_dataset.drop('visitId', axis=1, inplace=True)
test_dataset.drop('visitStartTime', axis=1, inplace=True)
test_dataset.drop('visits', axis=1, inplace=True)
test_dataset.drop('visitNumber', axis=1, inplace=True)
test_dataset.drop('isTrueDirect', axis=1, inplace=True)
test_dataset.drop('adContent', axis=1, inplace=True)
test_dataset.drop('networkDomain', axis=1, inplace=True)
test_dataset.drop('isVideoAd', axis=1, inplace=True)
test_dataset.drop('gclId', axis=1, inplace=True)
test_dataset.drop('slot', axis=1, inplace=True)
test_dataset.drop('adNetworkType', axis=1, inplace=True)
test_dataset.drop('keyword', axis=1, inplace=True)
test_dataset.drop('referralPath', axis=1, inplace=True)
test_dataset.drop('page', axis=1, inplace=True)
test_dataset.drop('newVisits', axis=1, inplace=True)
test_dataset.drop('bounces', axis=1, inplace=True)
test_dataset.drop('targetingCriteria', axis=1, inplace=True)
test_dataset.drop('socialEngagementType', axis=1, inplace=True)
test_dataset.drop('browserSize', axis=1, inplace=True)
test_dataset.drop('browserVersion', axis=1, inplace=True)
test_dataset.drop('flashVersion', axis=1, inplace=True)
test_dataset.drop('language', axis=1, inplace=True)
test_dataset.drop('mobileDeviceBranding', axis=1, inplace=True)
test_dataset.drop('mobileDeviceInfo', axis=1, inplace=True)
test_dataset.drop('mobileDeviceMarketingName', axis=1, inplace=True)
test_dataset.drop('mobileDeviceModel', axis=1, inplace=True)
test_dataset.drop('mobileInputSelector', axis=1, inplace=True)
test_dataset.drop('operatingSystemVersion', axis=1, inplace=True)
test_dataset.drop('screenColors', axis=1, inplace=True)
test_dataset.drop('screenResolution', axis=1, inplace=True)
test_dataset.drop('cityId', axis=1, inplace=True)
test_dataset.drop('longitude', axis=1, inplace=True)
test_dataset.drop('latitude', axis=1, inplace=True)
test_dataset.drop('networkLocation', axis=1, inplace=True)
test_dataset.drop('criteriaParameters', axis=1, inplace=True)


In [ ]:
test_dataset.head(5)

**Regardons les variables catégorielles que l'on va devoir changer**

In [ ]:
categorical_features_train_dataset = train_dataset.select_dtypes(include=[np.object])
categorical_features_train_dataset.columns

## LabelEncoder, OneHotEncoder

***to convert string categorical values to numerical values***

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
train_dataset['channelGrouping'] = labelencoder_X.fit_transform(train_dataset['channelGrouping'])
train_dataset['campaign'] = labelencoder_X.fit_transform(train_dataset['campaign'])
train_dataset['medium'] = labelencoder_X.fit_transform(train_dataset['medium'])
train_dataset['source'] = labelencoder_X.fit_transform(train_dataset['source'])
train_dataset['browser'] = labelencoder_X.fit_transform(train_dataset['browser'])
train_dataset['deviceCategory'] = labelencoder_X.fit_transform(train_dataset['deviceCategory'])
train_dataset['isMobile'] = labelencoder_X.fit_transform(train_dataset['isMobile'])
train_dataset['operatingSystem'] = labelencoder_X.fit_transform(train_dataset['operatingSystem'])
train_dataset['city'] = labelencoder_X.fit_transform(train_dataset['city'])
train_dataset['continent'] = labelencoder_X.fit_transform(train_dataset['continent'])
train_dataset['country'] = labelencoder_X.fit_transform(train_dataset['country'])
train_dataset['metro'] = labelencoder_X.fit_transform(train_dataset['metro'])
train_dataset['region'] = labelencoder_X.fit_transform(train_dataset['region'])
train_dataset['subContinent'] = labelencoder_X.fit_transform(train_dataset['subContinent'])
train_dataset['year'] = labelencoder_X.fit_transform(train_dataset['year'])
train_dataset['month'] = labelencoder_X.fit_transform(train_dataset['month'])
train_dataset['day'] = labelencoder_X.fit_transform(train_dataset['day'])

In [ ]:
train_dataset.head()


In [ ]:
test_dataset['channelGrouping'] = labelencoder_X.fit_transform(test_dataset['channelGrouping'])
test_dataset['campaign'] = labelencoder_X.fit_transform(test_dataset['campaign'])
test_dataset['medium'] = labelencoder_X.fit_transform(test_dataset['medium'])
test_dataset['source'] = labelencoder_X.fit_transform(test_dataset['source'])
test_dataset['browser'] = labelencoder_X.fit_transform(test_dataset['browser'])
test_dataset['deviceCategory'] = labelencoder_X.fit_transform(test_dataset['deviceCategory'])
test_dataset['isMobile'] = labelencoder_X.fit_transform(test_dataset['isMobile'])
test_dataset['operatingSystem'] = labelencoder_X.fit_transform(test_dataset['operatingSystem'])
test_dataset['city'] = labelencoder_X.fit_transform(test_dataset['city'])
test_dataset['continent'] = labelencoder_X.fit_transform(test_dataset['continent'])
test_dataset['country'] = labelencoder_X.fit_transform(test_dataset['country'])
test_dataset['metro'] = labelencoder_X.fit_transform(test_dataset['metro'])
test_dataset['region'] = labelencoder_X.fit_transform(test_dataset['region'])
test_dataset['subContinent'] = labelencoder_X.fit_transform(test_dataset['subContinent'])
test_dataset['year'] = labelencoder_X.fit_transform(test_dataset['year'])
test_dataset['month'] = labelencoder_X.fit_transform(test_dataset['month'])
test_dataset['day'] = labelencoder_X.fit_transform(test_dataset['day'])

In [ ]:
test_dataset.head()

In [ ]:
train_dataset.shape, test_dataset.shape

### Heatmap

Let's see correlation

In [ ]:
corelation_train = train_dataset.corr() 
sns.heatmap(corelation_train, square = True)

In [ ]:
corelation_test = test_dataset.corr() 
sns.heatmap(corelation_test, square = True)

### Split into train and test 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(train_dataset ,y, test_size=0.2)


### Premier modèle : Linear Regression

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, Y_train)

### Deuxième modèle : XGBRegressor

XGBoost est une bibliothèque optimisée de renforcement de gradient distribué conçue pour être très efficace, flexible et portable. Il implémente des algorithmes d'apprentissage automatique dans le cadre du Gradient Boosting. XGBoost fournit un boosting d'arbre parallèle (également connu sous le nom de GBDT, GBM) qui résout de nombreux problèmes de science des données de manière rapide et précise. 

In [ ]:
XGB_model = XGBRegressor()
XGB_model.fit(X_train, Y_train, eval_metric='rmse',verbose=130)

### Troisième modèle : Random forest

In [ ]:
RF_model = RandomForestRegressor(n_estimators = 100, random_state = 42)
RF_model.fit(X_train, Y_train)

### Quatrième modèle : LGBMRegressor


In [ ]:
LGB_model = lgb.LGBMRegressor()
LGB_model.fit(X_train, Y_train, eval_metric='rmse', verbose=100)

In [ ]:
y_pred_XGB = XGB_model.predict(X_test)
y_pred_LGB = LGB_model.predict(X_test)
y_pred_rf = RF_model.predict(X_test)
y_pred_lr = lin_reg.predict(X_test)

Remplacons les valeurs negatives avec 0

In [ ]:

y_pred_XGB[y_pred_XGB < 0] =0
y_pred_XGB
y_pred_LGB[y_pred_LGB < 0] =0
y_pred_LGB
y_pred_rf[y_pred_rf < 0] =0
y_pred_rf
y_pred_lr[y_pred_lr < 0] =0
y_pred_lr


In [ ]:
plt.scatter(Y_test, y_pred_lr)
plt.xlabel('True Values')
plt.ylabel('Predictions')

In [ ]:
plt.scatter(Y_test, y_pred_XGB)
plt.xlabel('True Values')
plt.ylabel('Predictions')

In [ ]:
plt.scatter(Y_test, y_pred_LGB)
plt.xlabel('True Values')
plt.ylabel('Predictions')

In [ ]:
plt.scatter(Y_test, y_pred_rf )
plt.xlabel('True Values')
plt.ylabel('Predictions')

## Evaluation des modèles utilisant le root mean squared error

In [ ]:
print("LinReg: sqrt  ", math.sqrt(mean_squared_error(Y_test, y_pred_lr)))
print("XGBoost: sqrt  ", math.sqrt(mean_squared_error(Y_test, y_pred_XGB)))
print("LGB: sqrt  ", math.sqrt(mean_squared_error(Y_test, y_pred_LGB)))
print("RandomForest: sqrt  ",math.sqrt(mean_squared_error(Y_test, y_pred_rf) ))

Prenons donc LGB car c'est lui qui nous donne le plus petit MSE

In [ ]:
final_result= LGB_model.predict(test_dataset)

In [ ]:
final_result.size

Voyons maintenant les caractéristiques importantes du modèle light gbm.


In [ ]:
fig, ax = plt.subplots(figsize=(12,18))
lgb.plot_importance(LGB_model, max_num_features=50, height=0.8, ax=ax)
ax.grid(False)
plt.title("Feature Important - LightGBM", fontsize=15)
plt.show()

In [ ]:
#Définissons toutes les valeurs négatives sur 0
final_result[final_result < 0] =0
final_result

In [ ]:
fullVisitorId_1 = test['fullVisitorId']

In [ ]:
fichier_csv = pd.DataFrame({'fullVisitorId': fullVisitorId_1,'PredictedLogRevenue':final_result},columns=['fullVisitorId','PredictedLogRevenue'])


In [ ]:
fichier_csv.head(1000)

In [ ]:
fichier_csv = fichier_csv.groupby('fullVisitorId').sum().reset_index()


In [ ]:
fichier_csv.to_csv(r"resultat_final.csv",index=False)


### FIN